In [1]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [2]:
# Clear any logs from previous runs
!rm -r /app/logs/

rm: cannot remove '/app/logs/wgangpModel/20240805-040909/train': Directory not empty
rm: cannot remove '/app/logs/wgangpModel/20240805-040909/validation': Directory not empty


In [3]:
import os,shutil,random
from sklearn.preprocessing import MinMaxScaler
import joblib
from IPython.display import clear_output
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
from sklearn.model_selection import train_test_split
import datetime as dt
import numpy as np
from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

2024-08-05 04:30:36.532677: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-05 04:30:36.538656: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-05 04:30:36.554043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 04:30:36.576254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 04:30:36.583097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 04:30:36.602193: I tensorflow/core/platform/cpu_feature_gu

In [4]:
#================== initialization ==================
#best LR = 1e-4,EPOCHS = 10,"discriminator_extra_steps":1
#best LR = 1e-4,EPOCHS = 15,"discriminator_extra_steps":1

currentTM=dt.datetime.now().strftime("%Y-%m-%dT%H%M%S")
PROJECT = "wgangpModel"
LATENT_DIM = 16
LR = 1e-4
EPOCHS = 10
BATCH_SIZE = 32

PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','TOTAL_VOLUME','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
MODEL_PATH = "./models/"
H5_PATH = './data/OptTrainData/'
STACK_DATA_SHAPE = np.empty((0,) + (16, len( SCALER_COL)  ) ) 

WANDB_LOG = True
RESUME = False
SUMMARY = True
log_dir = f"/app/logs/{PROJECT}/"+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
Scaler = joblib.load(SCALER_PATH )

#DISPLAY = ['map','summary',None]
DISPLAY = 'summary'


In [ ]:
##['UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','TOTAL_VOLUME','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
select_x = [i for i,c in  enumerate(SCALER_COL) if c in ['DTE','INTRINSIC_VALUE','C_VEGA','P_VEGA'] ]
select_y = [i for i,c in enumerate(SCALER_COL) if c in ['C_BID','C_ASK',  'P_BID',	'P_ASK'] ]

In [5]:
notes = f"""
test Run use_bias set false , no tranfrom
"""
import wandb
from wandb.integration.keras import WandbCallback
CONFIG = {
          "learning_rate": LR,
          "epochs": EPOCHS,
          "batch_size": BATCH_SIZE,
          "architecture": "wgangp",
          "dataset": "OptionsChaine",
          "generator_dense_units":[128,64,32],
          "generator_dropout_rate":0.2,
          "discriminator_dense_units":[32,64,128],
          "discriminator_dropout_rate":0.2,
          "use_bias":False,
          "use_dropout":True,
          "use_bn":True,
          "transform":True,
          "discriminator_extra_steps":1
          "x_col":select_x,
          "y_col":select_y,
           }

if WANDB_LOG :
    wandb.login()
    run = wandb.init(project=PROJECT, 
                     name=currentTM, 
                     config=CONFIG,
                     notes=notes
                    )

wandb: Currently logged in as: wasan-sinlapa. Use `wandb login --relogin` to force relogin


In [6]:
#=====================================================================

In [8]:
from src.wgangp.model import OptionChainGenerator
from src.wgangp.layer import generator, discriminator

model = OptionChainGenerator(
    discriminator = discriminator(
            input_shape= (16,len(select_y) ), 
            dense_units = CONFIG["generator_dense_units"], 
            dropout_rate= CONFIG["generator_dropout_rate"],
            use_bias=CONFIG["use_bias"],
            use_dropout=CONFIG["use_dropout"],
            use_bn=CONFIG["use_bn"]
           ), 
    generator = generator(
            input_dim = (16,len(select_y) ),
            output_dim = (16,len(select_x) ) ,
            dense_units = CONFIG["discriminator_dense_units"],
            dropout_rate= CONFIG["discriminator_dropout_rate"],
            use_bias=CONFIG["use_bias"],
            use_dropout=CONFIG["use_dropout"],
            use_bn=CONFIG["use_bn"]
           ),
    discriminator_extra_steps = CONFIG["discriminator_extra_steps"]
)


model.compile(
    d_optimizer = tf.optimizers.Adam(
    learning_rate=LR, beta_1=0.5, beta_2=0.9
    ),
    g_optimizer = tf.optimizers.Adam(
    learning_rate=LR, beta_1=0.5, beta_2=0.9
    )
)

In [9]:
################## show model ######################
if DISPLAY == 'map' :
    from tensorflow.keras.utils import model_to_dot
    from IPython.display import SVG, display
    
    def display_model(model, width=1024, height=512):
        dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
        svg_data = dot.create(prog='dot', format='svg').decode("utf-8")
        svg_html = f'<div style="width:{width}px;height:{height}px;">{svg_data}</div>'
        display(HTML(svg_html))

In [10]:
## Example usage:
## Display the generator model with reduced size
if DISPLAY == 'map' :
    display_model(model.generator, width, height=512)
if DISPLAY == 'summary' :
    model.generator.summary()

Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_x_data        │ (32, 16, 4)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (32, 4, 4, 4)     │          0 │ input_x_data[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv2D-32           │ (32, 4, 4, 32)    │      1,152 │ reshape[0][0]     │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (32, 4, 4, 32)    │        128 │ Conv2D-32[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (32, 4, 4, 128)   │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (32, 4, 4, 32)    │          0 │ leaky_re_lu_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv2D-64           │ (32, 4, 4, 64)    │     18,432 │ dropout_4[0][0]   │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (32, 4, 4, 64)    │        256 │ Conv2D-64[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (32, 4, 4, 64)    │          0 │ leaky_re_lu_1[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv2D-128          │ (32, 4, 4, 128)   │     73,728 │ dropout_5[0][0]   │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (32, 4, 4, 128)   │        512 │ Conv2D-128[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (32, 4, 4, 128)   │          0 │ leaky_re_lu_1[2]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (32, 2048)        │          0 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (32, 2048)        │          0 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (32, 64)          │    131,136 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Reshape-output      │ (32, 16, 4)       │          0 │ dense[0][0]       │
│ (Reshape)           │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 225,344 (880.25 KB)

 Trainable params: 224,896 (878.50 KB)

 Non-trainable params: 448 (1.75 KB)

In [11]:
if DISPLAY == 'map' :
    display_model(model.discriminator, width=2500, height=512)
if DISPLAY == 'summary' :
    model.discriminator.summary()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_y_data        │ (32, 16, 4)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (32, 16, 4, 1)    │          0 │ input_y_data[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (32, 20, 8, 1)    │          0 │ lambda[0][0]      │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (32, 20, 8, 128)  │      1,152 │ zero_padding2d[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (32, 20, 8, 128)  │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (32, 20, 8, 32)   │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (32, 20, 8, 128)  │          0 │ leaky_re_lu[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (32, 20, 8, 64)   │     73,728 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (32, 20, 8, 64)   │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (32, 20, 8, 64)   │          0 │ leaky_re_lu[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (32, 20, 8, 32)   │     18,432 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (32, 20, 8, 32)   │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (32, 20, 8, 32)   │          0 │ leaky_re_lu[2][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (32, 5120)        │          0 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (32, 5120)        │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv2D-logic        │ (32, 1)           │      5,121 │ dropout_3[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 99,329 (388.00 KB)

 Trainable params: 98,881 (386.25 KB)

 Non-trainable params: 448 (1.75 KB)

In [12]:
#================== loadmodel ====================

In [13]:
from tensorflow.keras.models import load_model
model_path = MODEL_PATH+f'{PROJECT}'
if not RESUME :
    if os.path.exists(model_path) :
        shutil.rmtree(model_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.generator.save(model_path+f'/'+f'generator.keras') 
    model.discriminator.save(model_path+f'/'+f'discriminator.keras') 
else:
    model.generator = load_model(model_path+'/'+f'generator.keras') 
    model.discriminator = load_model(model_path+'/'+f'discriminator.keras') 

In [14]:
# ================== train model ==================
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
#SET MODEL VAR
STACK_DATA = STACK_DATA_SHAPE 
#INIT MODEL VAR
STOP_MODEL = False

#set PartitionDate[:] for limit range
for partdate in PartitionDate[:] :
    clear_output(wait=False)
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    ###transform
    if CONFIG['transform'] :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)
        #! เช็ตปัญหาการ แปลง shape
        #inv.tran data ไม่ได้ค่าเดิม
    DATA = np.vstack((DATA ,STACK_DATA))

    if len(DATA) < 64 :
        #stack data
        STACK_DATA = np.vstack((STACK_DATA ,DATA))
    else: 
        STACK_DATA = np.empty((0,) + data_shape[1:] )
        X = DATA[:, :, select_x]  # เลือกข้อมูล select_x สำหรับ X
        Y = DATA[:, :, select_y]  # เลือกข้อมูล select_y เสำหรับ Y
        x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
        random.shuffle(PartitionDate)
        tf.keras.backend.clear_session() 
        history = model.fit(x_train , y_train, epochs=CONFIG['epochs'], batch_size=BATCH_SIZE, validation_data=(x_val, y_val) ,callbacks=[tensorboard_callback])
        if  np.isnan(  np.average( history.history['generator_loss'] )  ) or np.isnan(  np.average( history.history['discriminator_loss'] )  ):
            STOP_MODEL = True 
    
        if WANDB_LOG :
            LogKeys = history.history.keys()
            LogVal={}
            for k in LogKeys:  
                LogVal[k] = np.average(  history.history[k] )
            wandb.log(LogVal, commit=True)
    if STOP_MODEL :
        break
    
            
    model.generator.save(model_path+f'/'+f'generator.keras') 
    model.discriminator.save(model_path+f'/'+f'discriminator.keras') 
if WANDB_LOG : wandb.finish()

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 248ms/step - discriminator_loss: -0.1857 - generator_loss: -13.8122 - val_discriminator_loss: -2.1567 - val_generator_loss: -9.1525
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 213ms/step - discriminator_loss: -0.5425 - generator_loss: -14.6118 - val_discriminator_loss: -3.0100 - val_generator_loss: -9.2752
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 233ms/step - discriminator_loss: 0.1799 - generator_loss: -15.1429 - val_discriminator_loss: -2.9273 - val_generator_loss: -11.3066
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 234ms/step - discriminator_loss: 0.1504 - generator_loss: -15.4370 - val_discriminator_loss: -3.2714 - val_generator_loss: -11.8251
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 191ms/step - discriminator_loss: 0.0952 - generator_loss: -15.3564 - val_discriminator_loss: -3.1226 - val_generator_loss: -12.4682
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 238ms/step - discriminator_loss: 0.1529 - generator_loss: -14.9736 - val_discriminator_loss: 

discriminator_loss,▁▇▇█████████████████████████████████████
generator_loss,█▇▇▇▆▆▆▆▆▆▆▅▆▆▅▅▅▄▅▅▄▅▄▄▄▄▄▄▄▄▃▄▃▂▂▂▂▃▂▁
val_discriminator_loss,▇▆▆▅▅▅▅▅▅▅▅▅▆▅▅▅▅▆▆▆▅▄▆▆▅█▆▆▅▅▅▅▆▅▅▅▅▅▅▁
val_generator_loss,█▇██▇▇▇▆▇▆▇▆▇▇▅▅▆▄▆▅▅▆▄▄▃▃▄▄▅▅▃▄▃▂▂▁▂▂▁▁
discriminator_loss,-0.01798
generator_loss,-14.9895
val_discriminator_loss,-2.79776
val_generator_loss,-11.97063


In [15]:
`====================================================

SyntaxError: invalid syntax (2094489582.py, line 1)

In [ ]:
date_num = 1


partd = '2011-12'
with h5py.File(H5_PATH+partd+".h5", 'r') as f:
    d = f[partd][:]
d=Scaler.inverse_transform(
        d[10]
    )
d=pd.DataFrame(  
    d
,columns=SCALER_COL)
d

In [ ]:
======================== predict =========================

In [35]:
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    print(f"CONFIG['transform'] : {CONFIG['transform']}")
    if CONFIG['transform'] or True :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)

CONFIG['transform'] : True


In [36]:
col_x = [SCALER_COL[i] for i in select_x]
col_y = [SCALER_COL[i] for i in select_y]

In [37]:
X = DATA[:, :, select_x][:]
Y_real = DATA[:, :, select_y][:]
#===========
# X = x_train
# Y_real = y_train
# #===========
dfX = pd.DataFrame(
    X[:1].reshape(16, len(select_x)), 
    columns=col_x)
#print(dfX)

In [38]:
genVal = model.generator(X) 

In [39]:
dfY = pd.DataFrame(
    genVal.numpy()[:1].reshape(16, len(select_y)), 
    columns=col_y)
#print(dfY)

In [40]:
SUMDF = pd.concat([dfX, dfY],axis=1)

In [41]:
resultDF = pd.DataFrame([])
rm_col = []
for i in SCALER_COL:
    if i in SUMDF.columns:
         resultDF[i] = SUMDF[i]
    else:
        rm_col += [i]
        resultDF[i] = [1e-8]*16


In [42]:
if CONFIG['transform'] :
    decode_transformed=Scaler.inverse_transform(
        resultDF
    )
    decode_transformed=pd.DataFrame(  
        decode_transformed
    ,columns=SCALER_COL).drop(columns=rm_col)
    decode_transformed.loc[decode_transformed['DTE'] < 1e-8] = 0
    decode_transformed[col_y] = decode_transformed[col_y].round(2)

In [43]:
decode_transformed

,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,P_BID,P_ASK
0,-1.0,4.0,0.04732,4.749000e-02,9.26,8.80,3.64,3.28
1,-1.5,4.0,0.04345,4.407000e-02,8.97,8.30,3.74,3.54
2,-2.0,4.0,0.03796,3.899000e-02,9.64,8.31,4.09,3.68
3,-2.5,4.0,0.03282,3.237000e-02,9.56,8.97,4.68,4.17
4,-3.0,4.0,0.02610,2.809000e-02,8.88,8.54,5.44,5.38
5,-3.5,4.0,0.01948,2.434000e-02,8.91,9.06,5.83,5.61
6,-4.0,4.0,0.01410,1.613000e-02,9.79,9.33,6.11,5.32
7,-4.5,4.0,0.00999,1.311000e-02,9.03,9.44,5.82,5.41
8,-5.0,4.0,0.00590,1.590000e-02,9.11,8.93,6.02,5.85
9,-5.5,4.0,0.00493,1.278000e-02,8.32,8.26,6.14,6.29


In [44]:
======================= real data ==============================

SyntaxError: invalid syntax (2694477771.py, line 1)

In [45]:
if CONFIG['transform'] :
    realdecode_transformed=Scaler.inverse_transform(
        DATA[0]
    )
    realdecode_transformed=pd.DataFrame(  
        realdecode_transformed
    ,columns=SCALER_COL).drop(columns=rm_col)
    realdecode_transformed.loc[realdecode_transformed['DTE'] <= 1e-8] = 0
    

In [46]:
realdecode_transformed

,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,P_BID,P_ASK
0,-1.0,4.0,0.04732,4.749000e-02,6.700000e-01,7.000000e-01,1.61,1.67
1,-1.5,4.0,0.04345,4.407000e-02,4.900000e-01,5.200000e-01,1.90,1.96
2,-2.0,4.0,0.03796,3.899000e-02,3.400000e-01,3.500000e-01,2.28,2.35
3,-2.5,4.0,0.03282,3.237000e-02,2.300000e-01,2.500000e-01,2.62,2.69
4,-3.0,4.0,0.02610,2.809000e-02,1.400000e-01,1.500000e-01,3.06,3.16
5,-3.5,4.0,0.01948,2.434000e-02,9.000000e-02,1.000000e-01,3.52,3.60
6,-4.0,4.0,0.01410,1.613000e-02,6.000000e-02,6.000000e-02,3.91,4.04
7,-4.5,4.0,0.00999,1.311000e-02,3.000000e-02,5.000000e-02,4.39,4.50
8,-5.0,4.0,0.00590,1.590000e-02,2.000000e-02,2.000000e-02,4.94,5.03
9,-5.5,4.0,0.00493,1.278000e-02,9.999994e-09,2.000000e-02,5.39,5.48


In [31]:
9.999994e-09

9.999994e-09

In [ ]:
======================= _compute_loss ==============================

In [ ]:
#generated_data = [c_bid, c_ask, c_volume, p_bid, p_ask, p_volume]
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]
generated_data = decode_data[3:]
z_mean    = z_mean
z_log_var = log_var
Y_real    = DATA[:, :, 3:][:1]

In [ ]:
z_mean

In [ ]:
for  col,genData in zip(colList,generated_data):
    print( colList.index(col),col )

In [ ]:
subtract_genData = genData - tf.cast(tf.expand_dims(Y_real[:, :, colList.index(col)], axis=-1)
        , tf.float32) 

In [ ]:
reconstruction_values_total = []
reconstruction_values_total.append( tf.reduce_mean( tf.square(subtract_genData)   ) )

In [ ]:
log_var = tf.clip_by_value(log_var, -1.0, 1.0)
kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(z_mean) - tf.exp(log_var), axis=-1)

In [ ]:
tf.reduce_mean(reconstruction_values_total + kl_loss)

In [ ]:
========== kiras vae origi

In [ ]:
Y_real[0][0]

In [ ]:
 tf.concat(decode_data, axis=-1).numpy()[0][0]

In [ ]:
Y_real[0] -  tf.concat(decode_data, axis=-1).numpy()

In [ ]:
features_loss = tf.reduce_mean(
    tf.reduce_sum(
        tf.keras.losses.mean_squared_error(Y_real, tf.concat(decode_data, axis=-1)),
        axis=(1),
    )
)

In [ ]:
features_loss

In [ ]:
# features_loss = tf.reduce_mean(
#     tf.reduce_sum(
#         keras.losses.categorical_crossentropy(features_real, features_gen),
#         axis=(1),
#     )
# )
# kl_loss = -0.5 * tf.reduce_sum(
#     1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), 1
# )
# kl_loss = tf.reduce_mean(kl_loss)

# property_loss = tf.reduce_mean(
#     keras.losses.binary_crossentropy(qed_true, qed_pred)
# )

# graph_loss = self._gradient_penalty(graph_real, graph_generated)

# return kl_loss + property_loss + graph_loss + adjacency_loss + features_loss

In [ ]:
======================= inverse_transform ========================

In [ ]:
#add 0
decode_data = [tf.zeros([1, 32, 1])]*3 + decode_data

In [ ]:
invert_decode = Scaler.inverse_transform(
    np.array([d.numpy().reshape(-1) for d in decode_data]).transpose()
    ) 

In [ ]:
pd.DataFrame(
    invert_decode[:,3:], 
    columns=SCALER_COL[3:])

In [ ]:
=====================================================================